## Imports

In [6]:
import os 
import pandas as pd

## Carregando os dados

In [8]:
dataset_path = 'dataset/'
dataset_name = 'dataset.csv'

dataset = pd.read_csv(os.path.join(dataset_path, dataset_name))
dataset

,sentence,category
0,"Auxílio-Doença Previdenciário, Benefícios em E...",orgão público
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",finanças
2,Então encontraremos na próxima aula.,educação
3,Veja os resultados da categoria de ofertas do ...,indústrias
4,"Além disso, a embalagem é reutilizável e 100% ...","indústrias,varejo"
...,...,...
516,"Selecione o local de estudo, curso sem encontr...",educação
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,"educação,orgão público"
518,Empresas e órgãos públicos,orgão público
519,DGE – Departamento de Gestão Estratégica Metas...,orgão público


## Exploratória

In [25]:
dataset.iloc[517]


sentence    ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...
category                               educação,orgão público
Name: 517, dtype: object

In [9]:
texto = 'sentence'
setor = 'category'

dataset[setor].value_counts()

category
orgão público               140
educação                    107
indústrias                   89
varejo                       85
finanças                     54
finanças,varejo              13
educação,orgão público        9
indústrias,varejo             7
educação,indústrias           5
finanças,orgão público        4
finanças,indústrias           3
indústrias,orgão público      2
educação,finanças             2
varejo,indústrias             1
Name: count, dtype: int64

Observando a frequência com que os setores são atribuídos aos textos, fica claro que o 'multi-labeling' não é tão presente, ou seja,
existem bem mais textos classificados à apenas um setor. 

Nesse momento, é importante decidir qual será a abordagem da modelagem. A priori, tendo a acreditar que modelar um multi-label não vá ser eficiente dado esse 'desbalanceamento'.

Uma alternativa é associar cada texto à um único setor.
- **E aqueles que são associados a dois setores?**

    Estes serão duplicados e associados aos dois setores. 
    
    Exemplo:
    
     'ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS EM VALPARAÍSO.' -> 'educação'   
     'ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS EM VALPARAÍSO.' -> 'orgão público'

    Assim, quando o modelo receber um input que se referir tanto ao setor 'educação' quanto ao setor 'orgão público', ele tenderá a dar pesos mais altos para essas duas classes. 

- **O modelo não vai ficar confuso tendo duas labels para o mesmo texto?**

    Essa abordagem de fato pode atrapalhar o aprendizado do modelo e diminuir a acurácia, no entanto, parte-se do pressuposto de que o modelo tenderá a atribuir maiores pesos para os textos relacionados à um setor em específico.

In [14]:
dataset[texto].sample(5).tolist()

['Veja como foi a trajetória de debates do Projeto de Regularização Imobiliária na Câmara Municipal até ele se tornar Lei.',
 'O dólar americano continua sendo a moeda constitutiva de mais de 70% das reservas internacionais.',
 'Relações Humanas, carga horária total do módulo 14hs.',
 'Requisitos de segurança do carro Camaro Chevrolet.',
 'PROJETO DE LEI Nº 260/2020.']